In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [17]:
# Load Twitter data from a CSV file
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/Twitter_Data.csv')

In [18]:
data.shape

(162980, 2)

In [19]:
# Check the first few rows of the dataset
data.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [20]:
# Dataset description
data.describe()

,category
count,162973.000000
mean,0.225436
std,0.781279
min,-1.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [21]:
null_values = data.isnull().sum()

In [22]:
data.isnull()

,clean_text,category
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
162975,False,False
162976,False,False
162977,False,False
162978,False,False


In [23]:
# Fill missing values in 'clean_text' column with a placeholder
data['clean_text'].fillna("Placeholder", inplace=True)

In [24]:
# Preprocess the data
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
X = data['clean_text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=128, truncation=True))
y = data['category']

In [25]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load pre-trained BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [27]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train.tolist(), y_train.tolist())).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test.tolist(), y_test.tolist())).batch(32)

ValueError: ignored

In [ ]:
# Train the model
history = model.fit(train_dataset, epochs=3, validation_data=test_dataset)

In [ ]:

# Evaluate the model
y_pred = model.predict(test_dataset)
y_pred_labels = np.argmax(y_pred[0], axis=1)
accuracy = accuracy_score(y_test, y_pred_labels)

print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred_labels))

# Plot accuracy over epochs
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()